# DOP

Dillution of Precision (DOP) describes the effect of satellite geometry on the 
user's positioning accuracy. 
DOP can refer to multiple ideas:

1. DOP in the horizontal or vertical direction,
2. DOP on the time uncertainty,
3. The full DOP matrix, or
4. DOP in a particular direction.

Here we show how to make use of existing functionality. 
For more details on the underlying math, please see the [Navipedia page on 
positioning error](https://gssc.esa.int/navipedia/index.php?title=Positioning_Error).

In [ ]:
import gnss_lib_py as glp
import numpy as np

# A library for url downloads that works regardless of `wget` command
import urllib.request

As an example, we can load in data from the 2022 Google Smartphone Decimeter 
Challenge

In [ ]:
glp.make_dir("../data")
urllib.request.urlretrieve("https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/data/unit_test/google_decimeter_2022/device_gnss.csv", "../data/device_gnss.csv")

navdata = glp.AndroidDerived2022("../data/device_gnss.csv")

Since we already have elevation and azimuth data available to us, we can simply
call the `get_dop` function, which will only return the HDOP and VDOP values,
by default.
HDOP corresponds to the dilution of precision in the horizontal direction
(i.e., East and North plane), and VDOP corresponds to the dilution of precision
in the vertical direction (i.e., the Up axis).

In [ ]:
dop_navdata = glp.get_dop(navdata)
print(dop_navdata)

Some applications may care about the dilution of precision in time (TDOP), but
may not be interested in the dilution of precision in position. 
Simply pass this information to the parser.

In [ ]:
dop_navdata = glp.get_dop(navdata, HDOP=False, VDOP=False, TDOP=True)
print(dop_navdata)

Below we illustrate all supported DOP types. The full DOP matrix (in ENU) is 

$$Q = 
\begin{bmatrix}
    q_{ee} & q_{en} & q_{eu} & q_{et} \\
    q_{ne} & q_{nn} & q_{nu} & q_{nt} \\
    q_{ue} & q_{un} & q_{uu} & q_{ut} \\
    q_{te} & q_{tn} & q_{tu} & q_{tt} \\
\end{bmatrix}$$

The matrix is symmetric (i.e., $q_{en} = q_{ne}$). 
Often the elements along the diagonal are of primary interest
$$Q = 
\begin{bmatrix} 
    \text{EDOP}^2 & \cdot & \cdot & \cdot \\
    \cdot & \text{NDOP}^2 & \cdot & \cdot \\
    \cdot & \cdot & \text{VDOP}^2 & \cdot \\
    \cdot & \cdot & \cdot & \text{TDOP}^2
\end{bmatrix}$$


To store the dop matrix $Q$ in `dop_navdata`, the upper triangle is
splatted across columns to enable fast storage and access in the navdata using
numpy.

In [ ]:
dop_navdata = glp.get_dop(navdata, GDOP= True, HDOP=True, VDOP=True,
                          PDOP=True, TDOP=True, dop_matrix=True)
print(dop_navdata)

We can recover the unsplatted versions of the DOP matrix as needed if we 
loop through time.

In [ ]:
for timestamp, _, dop_navdata_subset in glp.loop_time(dop_navdata, 'gps_millis'):

    labels = glp.get_enu_dop_labels()

    dop_matrix_splat = np.array(
        [dop_navdata_subset[f'dop_{label}'] for label in labels])

    print(f"At time {timestamp} the DOP matrix is")
    print(glp.unsplat_dop_matrix(dop_matrix_splat))

Lastly, we can compute the contribution in a particular direction with numpy.

In [ ]:
direction_of_interest = np.array([-1, 1, 0, 0])
# Normalize the direction of interest
direction_of_interest = direction_of_interest / np.linalg.norm(direction_of_interest)

for timestamp, _, dop_navdata_subset in glp.loop_time(dop_navdata, 'gps_millis'):

    labels = glp.get_enu_dop_labels()

    dop_matrix_splat = np.array(
        [dop_navdata_subset[f'dop_{label}'] for label in labels])
    dop_matrix_unsplat = glp.unsplat_dop_matrix(dop_matrix_splat)

    dop_in_direction = np.sqrt(
        direction_of_interest @ dop_matrix_unsplat @ direction_of_interest)

    print(f"At time {timestamp} the DOP in the direction of interest is {dop_in_direction}")